In [1]:
# Dependencies
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps

# Google API Key
from config import gkey
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# read CSV from WeatherPy
df = pd.read_csv("Resources/WeatherPy.csv")
df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature (F),Wind Speed (mph),Cloudiness %,Humidity %,Date
0,0,Port Hardy,CA,50.70,-127.42,70.07,4.70,75,52,1596346233
1,1,Okhotsk,RU,59.38,143.30,64.96,8.21,1,55,1596346311
2,2,Najrān,SA,17.49,44.13,84.47,11.41,22,62,1596346311
3,3,Ketchikan,US,55.34,-131.65,58.19,5.82,90,100,1596346052
4,4,Deputatsky,RU,69.30,139.90,63.64,9.31,29,47,1596346311


In [3]:
# create coordinates
coordinates = list(zip(df.Latitude, df.Longitude))
humidity=list(df["Humidity %"])


In [4]:
# Create a temperature Heatmap layer
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig
# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [5]:
ideal_temp_df=df[df['Temperature (F)'] < 80] 
ideal_temp_df_2=ideal_temp_df[ideal_temp_df['Temperature (F)'] >75] 
ideal_cloud=ideal_temp_df_2[ideal_temp_df_2['Cloudiness %']<10]
ideal_places=ideal_cloud[ideal_cloud["Humidity %"]>50]
ideal_places

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature (F),Wind Speed (mph),Cloudiness %,Humidity %,Date
96,96,Pascagoula,US,30.37,-88.56,77.25,7.72,1,88,1596346327
222,222,Kiryat Gat,IL,31.61,34.76,77.81,3.00,0,57,1596346348
287,287,São Filipe,CV,14.90,-24.50,77.90,1.12,0,83,1596346360
354,354,Cehegín,ES,38.09,-1.80,78.04,1.01,0,63,1596346370
531,531,Agropoli,IT,40.35,15.00,77.31,2.39,0,74,1596346401
559,559,Kiryat Gat,IL,31.61,34.76,77.81,3.00,0,57,1596346348
560,560,Bayir,TR,37.27,28.22,77.27,2.24,0,57,1596346406


In [9]:
hotel_df=ideal_places[["City","Country", "Latitude", "Longitude"]]
hotel_new=hotel_df.reset_index()
hotel_new2=hotel_new[["City", "Country", "Latitude", "Longitude"]]
hotel_new2

,City,Country,Latitude,Longitude
0,Pascagoula,US,30.37,-88.56
1,Kiryat Gat,IL,31.61,34.76
2,São Filipe,CV,14.90,-24.50
3,Cehegín,ES,38.09,-1.80
4,Agropoli,IT,40.35,15.00
5,Kiryat Gat,IL,31.61,34.76
6,Bayir,TR,37.27,28.22


In [11]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_new2.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params).json()

    try:
        hotel_new2.loc[index, "Hotel"] = response["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_new2
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
# print(response.url)


,City,Country,Latitude,Longitude,Hotel
0,Pascagoula,US,30.37,-88.56,Regency Inn
1,Kiryat Gat,IL,31.61,34.76,רגע בנחת
2,São Filipe,CV,14.90,-24.50,Tortuga B&B
3,Cehegín,ES,38.09,-1.80,Apartamentos Begastri
4,Agropoli,IT,40.35,15.00,B&B Baia di Trentova
5,Kiryat Gat,IL,31.61,34.76,רגע בנחת
6,Bayir,TR,37.27,28.22,Buban Otel


In [22]:
coordinates_2=list(zip(hotel_new2.Latitude, hotel_new2.Longitude))

fig
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates_2)
# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…